# Initialiazation

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

# Read Bronze table

# Silver Transformation

In [0]:
df = spark.table("workspace.bronze.erp_px_cat_g1v2")

## Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

## Maintenance flag normalization to booloean

In [0]:
df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
    .when(F.upper(col("maintenance")) == "NO", F.lit(False))
    .otherwise(None)
)

## Rename column

In [0]:
RENAME_MAP = {
    "ID": "category_id",
    "CAT":"category",
    "SUBCAT":"subcategory",
    "MAINTENANCE":"maintenance_flag"
}

for old_name, new_name in RENAME_MAP.items():
  df = df.withColumnRenamed(old_name, new_name)

## Sanity check

In [0]:
# df.limit(10).display()

## Write to Silver table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_px_cat_g1v2")

## Sanity check

In [0]:
%sql
SELECT *
FROM workspace.silver.erp_px_cat_g1v2 LIMIT 10